In [ ]:
credentials_path = '../credentials.json'

In [ ]:
# First, let's clean up any existing Firebase instances
import firebase_admin
if firebase_admin._apps:
    app = firebase_admin.get_app()
    firebase_admin.delete_app(app)

# Now install and import pyfiresql
!pip install pyfiresql pandas

# Import required libraries
from firesql.firebase import FirebaseClient
from firesql.sql.sql_fire_client import FireSQLClient
from firesql.sql import FireSQL
import pandas as pd

def initialize_firesql():
    """Initialize FireSQL with proper cleanup"""
    try:
        # Clean up any existing Firebase instances
        import firebase_admin
        if firebase_admin._apps:
            app = firebase_admin.get_app()
            firebase_admin.delete_app(app)
            
        # Initialize FireSQL
        client = FirebaseClient()
        client.connect(credentials_json=credentials_path)
        sql_client = FireSQLClient(client)
        fire_sql = FireSQL()
        return fire_sql, sql_client
    except Exception as e:
        print(f"Error initializing FireSQL: {str(e)}")
        return None, None

def run_firesql_query(sql_client, fire_sql, query):
    """Run a FireSQL query and return results as a pandas DataFrame"""
    try:
        results = fire_sql.execute(sql_client, query)
        if results:
            return pd.DataFrame(results)
        return pd.DataFrame()
    except Exception as e:
        print(f"Error executing query: {str(e)}")
        return None

# Initialize FireSQL
fire_sql, sql_client = initialize_firesql()

if fire_sql and sql_client:
    # Example query - adjust collection name as needed
    query = "SELECT * FROM users;"
    df = run_firesql_query(sql_client, fire_sql, query)
    
    if df is not None and not df.empty:
        print(f"Found {len(df)} users:")
        display(df)
    else:
        print("No results returned or error occurred")
else:
    print("Failed to initialize FireSQL client")
    print("\nTroubleshooting tips:")
    print("1. Try restarting the kernel and running this cell again")
    print("2. Check that 'users' is the correct collection name")
    print("3. Verify your Firestore security rules allow read access")

In [ ]:
def run_firesql(query, limit=None):
    """
    Execute a FireSQL query and return results as a pandas DataFrame.
    
    Args:
        query (str): The SQL query to execute
        limit (int, optional): Maximum number of results to return
    
    Returns:
        pandas.DataFrame: Query results, or None if there was an error
    """
    # Initialize FireSQL if not already done
    if 'fire_sql' not in globals() or 'sql_client' not in globals():
        global fire_sql, sql_client
        fire_sql, sql_client = initialize_firesql()
    
    if not fire_sql or not sql_client:
        print("Failed to initialize FireSQL client")
        return None
    
    try:
        # Execute the query
        results = fire_sql.execute(sql_client, query)
        
        if not results:
            print("No results returned")
            return pd.DataFrame()
            
        df = pd.DataFrame(results)
        
        # Apply limit if specified
        if limit and len(df) > limit:
            print(f"Returning first {limit} of {len(df)} results")
            return df.head(limit)
            
        print(f"Found {len(df)} results")
        display(df)
        
    except Exception as e:
        print(f"Error executing query: {str(e)}")
        return None

# Example usage:
# Get all users
# users_df = run_firesql("SELECT * FROM users;")

# Get first 10 active users
# active_users = run_firesql("SELECT * FROM users WHERE status = 'active';", limit=10)

# Get specific fields
# user_emails = run_firesql("SELECT email, name FROM users;")

In [ ]:
# Example query
user_exists = run_firesql("""
    SELECT *
    FROM users 
    WHERE email = 'example@email.com' 
    ;
""")